In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import tidytcells as tt
from scipy import optimize
from scipy.stats import pearsonr
from scipy.spatial.distance import pdist
import Levenshtein 
import scipy.stats as stats
import random
def pflag(p,dt):
    return int(random.random()<p*dt)
def Tinf(aT,bT,T0,K,t,dt):
    q = K/(1e-5)
    ind = round(q*T0/dt)
    N = np.exp(aT*t)
    N[ind+1:] = N[ind+1]*np.exp(-bT*(t[ind+1:]-t[ind])) 
    return N

In [2]:
# simulation parameters
dt = 1e-1
Dend = 30*24
# T cells 
NT = 500 # number of T cells
aT = 1/7.9
bT = 1/55
T0 = 2.5*24 # duration of proliferation
sig = 0.20

mode = 3

if mode == 0:
    Nvec = np.ones(NT)*10
    Kvec = np.ones(NT)*(1e-5)
    trt = 'neu'
if mode == 1:
    Nvec = np.ones(NT)*10
    Kvec = np.abs((1e-5)*np.random.normal(1,sig,NT))
    trt = 'aff'
if mode == 2:
    Nvec = np.round(10**np.random.normal(1,sig,NT))
    Kvec = np.ones(NT)*(1e-5)
    trt = 'num'
if mode == 3:
    Nvec = np.round(10**np.random.normal(1,sig,NT))
    Kvec = np.abs((1e-5)*np.random.normal(1,sig,NT))
    trt = 'both'
    
r0 = 1e-1
# Virus
aV = 1e-1
gV = 1e-5
# pMHC
kappa = 1e-1
ST = 1e4
###########
M = round(Dend/dt) + 1
t = np.array(list(range(M)))*dt
# pre-allocate #
T = np.zeros([NT,M]); 
V = np.zeros(M); V[0] = 1e-9

In [ ]:
# stochastic model

collector = []
collector_x = []
for i in range(M-1):
    # if not np.mod(i,round(M/10)):
    #     print(str(round(100*(i/M))))
    ###### input
    Tin = T[:,i]
    Vin = V[i]
    ######
    Sbar = ST*Vin/(Vin+kappa)
    p = r0*Sbar*Nvec/((Sbar+1/Kvec)*(1+np.sum(Nvec/(Sbar+1/Kvec))))
    Act_flag = [pflag(r0*x,dt) for x in p]
    ######
    for j, a in enumerate(Act_flag):
        if a:
            collector.append(t[i])
            collector_x.append(j)
            Tvec = Tinf(aT,bT,T0,Kvec[j],t,dt)
            T[j,i:] = T[j,i:] + Tvec[:len(T[j,i:])]
    V[i+1] = V[i] + dt*(aV*Vin*(1-Vin)-gV*Vin*sum(Tin))
    if V[i+1] < 1e-9:
        V[i+1] = 0
    for j, y in enumerate(T[:,i+1]):
        if y < 1:
            T[j,i+1] = 0

##############
#############
##############
#############
##############
Tstar = []
fmax = []
fig, ax1 = plt.subplots(figsize=(2,1))
for y in T:
    ax1.plot(t/24,np.log10(y),'k',linewidth=0.1)
    qr = [t[i]/24 for i, z in enumerate(y) if z == np.max(y)]
    Tstar.append(qr[0])
    fmax.append([z for i, z in enumerate(y) if z == np.max(y)])
ax1.set_xlabel(r'$t$, days')
ax1.set_ylabel(r'log $T$', color='k')
ax1.tick_params(axis='y', labelcolor='k')
ax1.set_ylim([2,5])
ax1.set_yticks([2,3,4,5])
ax1.set_xlim([0,20])
ax1.set_xticks([0,10,20])

ax2 = ax1.twinx()

ax2.plot(t/24,np.log10(V), color='red')
ax2.set_ylabel(r'log $V$', color='red')
ax2.tick_params(axis='y', labelcolor='red')
ax2.set_ylim([-7,1])
ax2.set_yticks([-6,-3,0])

fig.savefig('figs/Traj_and_viral_lvls_'+trt+'.svg')

##########################

ed = np.linspace(8,14,12)
x = ed[1:] - 0.5*(ed[1]-ed[0])
y = []
y_err = []
for i in range(len(x)):
    temp = [np.log10(fmax[j]) for j, z in enumerate(Tstar) if z >= ed[i] and z < ed[i+1]]
    if len(temp) <= 2:
        y.append(np.nan)
        y_err.append(np.nan)
    if len(temp) > 2:
        y.append(np.mean(temp))
        y_err.append(np.std(temp)/np.sqrt(len(temp)-1))
    
plt.figure(figsize=(1,1))
plt.errorbar(x,y,y_err,fmt='k.',markersize=4,capsize=2)
plt.ylim([2,5])
plt.yticks([2,3,4,5])
plt.xlim([8,14])
plt.xticks([8,10,12,14])
plt.xlabel(r'$t^*$, days')
plt.ylabel(r'log $T^*$')
##########################
plt.savefig('figs/logTvtStar_'+trt+'.svg')
##########################

/var/folders/zm/c02xmwfx58l3xfx_hgwyrdgw0000gn/T/ipykernel_80920/1562030378.py:38: RuntimeWarning: divide by zero encountered in log10
  ax1.plot(t/24,np.log10(y),'k',linewidth=0.1)


In [ ]:
ed = np.linspace(8.1,13.2,10)
x = ed[1:] - 0.5*(ed[1]-ed[0])
y = []
y_err = []
for i in range(len(x)):
    temp = [np.log10(fmax[j]) for j, z in enumerate(Tstar) if z >= ed[i] and z < ed[i+1]]
    if len(temp) <= 2:
        y.append(np.nan)
        y_err.append(np.nan)
    if len(temp) > 2:
        y.append(np.mean(temp))
        y_err.append(np.std(temp)/np.sqrt(len(temp)-1))
    
plt.figure(figsize=(1,1))
plt.errorbar(x,y,y_err,fmt='k.',markersize=4,capsize=2)
plt.ylim([2,5])
plt.yticks([2,3,4,5])
plt.xlim([8,14])
plt.xticks([8,10,12,14])
plt.xlabel(r'$t^*$, days')
plt.ylabel(r'log $T^*$')
plt.savefig('figs/logTvtStar_'+trt+'.svg')